# Broadcast

### But : Faire un broadcast, départ décalé 


#### Informations générales

In [2]:
experiment_name = "Time_Bcast"

In [3]:
%%bash -s
git log -n 1
python --version

commit 70e99ea3fdd4a04864c8c11ad4cca167f3771112
Author: Najwa Ez Zine <nezzine@fnantes-stretch>
Date:   Thu May 31 12:29:29 2018 +0200

    //


Python 2.7.13


#### Préparation de l'environnement

In [4]:
%%bash -s
pip3 install --user execo
pip3 install --user requests

  Running setup.py bdist_wheel for execo: started
  Running setup.py bdist_wheel for execo: finished with status 'done'
  Stored in directory: /home/nezzine/.cache/pip/wheels/8b/bd/44/454509c09ace69ed15ae84bdf707d305bff5e4468213f1df48
Successfully built execo


In [5]:
import execo
from execo import *
from execo_g5k import *
from execo_engine import *
import os
import sys
import json
import time
import datetime
import re
from shutil import copy
from subprocess import check_output

##### Réservation noeuds

In [9]:
# Nom du job
jobname = ' '
# Nombre de noeuds
nodecount = 4
# Temps réservation
walltime = "3:0:0" 

# filters out Nantes's econome cluster
resources_selection = "-p \"cluster in ('ecotype','parasilo','grisou')\"" 

# Nancy
site = "rennes"

In [12]:
jobs = get_current_oar_jobs()
jobid = None
waiting_jobs = []
while jobs:
    j, site = jobs.pop()
    info = get_oar_job_info(j, site)
    if info['name'] == jobname:
        if info['state'] == 'Running':
            jobid = j
            print("A {} job is already running, using it. jobid is {}".format(jobname, jobid))
            break
        else:
            waiting_jobs.append(j)
if not jobid and not waiting_jobs:
    jobspec = OarSubmission(resources="/cluster=1/nodes={}".format(nodecount), walltime=walltime,
                            additional_options=resources_selection, job_type="deploy", name=jobname)
    jobid, _ = oarsub([(jobspec, site)]).pop()
    print("New job submitted, jobid is {}".format(jobid))
elif not jobid:
    print("One or more {} jobs exist ({}) but are not running.\n"
          " Connect to the frontend to see what is happening, and/or run the cell again.".format(
          jobname, ", ".join([str(j) for j in waiting_jobs])))

New job submitted, jobid is 1014337


On les range

In [13]:
nodes = get_oar_job_nodes(jobid)
nodes.sort(key=lambda n: n.address)
nodes

[Host('parasilo-26.rennes.grid5000.fr'),
 Host('parasilo-27.rennes.grid5000.fr'),
 Host('parasilo-28.rennes.grid5000.fr'),
 Host('parasilo-9.rennes.grid5000.fr')]

##### Déploiement de l'environnement

In [14]:
force_redeploy = False # set to True to force redeploying the OS on the nodes in the deployment section
environment_dsc_file = 'src/debian9-x64-bigdata-tutorial.yaml' # filename of the kadeploy environment file (YAML)

In [15]:
deployment = Deployment(hosts=nodes, env_file = os.path.abspath(environment_dsc_file),
                        other_options="-r ext4 --no-debug-mode")

deploy_ok, deploy_failed = deploy(deployment, check_deployed_command=not force_redeploy,
                              stdout_handlers=[sys.stdout],
                              stderr_handlers=[sys.stderr])

rennes: Deployment #D-f0e20d50-3308-4b23-bbd8-b6b54241becb started
rennes: Grab the key file /home/nezzine/.ssh/authorized_keys
rennes: Grab the tarball file http://public.nancy.grid5000.fr/~pneyron/debian9-x64-bigdata-tutorial.tar.gz
rennes: Grab the postinstall file server:///grid5000/postinstalls/g5k-postinstall.tgz
rennes: Launching a deployment on parasilo-[9,26-28].rennes.grid5000.fr
rennes: Performing a Deploy[SetDeploymentEnvUntrusted] step
rennes:   switch_pxe
rennes:   reboot
rennes:    * Performing a soft reboot on parasilo-[9,26-28].rennes.grid5000.fr
rennes:   wait_reboot
rennes:   send_key_in_deploy_env
rennes:   create_partition_table
rennes:   format_deploy_part
rennes:   mount_deploy_part
rennes:   format_tmp_part
rennes:   format_swap_part
rennes: End of step Deploy[SetDeploymentEnvUntrusted] after 208s
rennes: Performing a Deploy[BroadcastEnvKascade] step
rennes:   send_environment
rennes:    * Broadcast time: 71s
rennes:   manage_admin_post_install
rennes:   manage_

rennes: Warning: Permanently added 'rennes.grid5000.fr,172.16.111.106' (ECDSA) to the list of known hosts.
rennes: Connection to rennes.grid5000.fr closed.


#### Test : commande simple 'ls'

In [ ]:
# execo.action.Remote(cmd,hosts,connection_params,process_args)
Remote_test = execo.action.Remote(cmd='ls',hosts=nodes,connection_params=None, process_args=None)
Remote_test.run().ok

#### Installation environnement sur noeud

In [16]:
Remote_install = execo.action.Remote(cmd='apt-get install g++ libboost-all-dev && wget http://gforge.inria.fr/frs/download.php/latestfile/8/SimGrid-3.18.tar.gz && tar -xvf SimGrid-3.18.tar.gz && cd SimGrid-3.18 && cmake -DCMAKE_INSTALL_PREFIX=/usr/local -Denable_smpi=on -Denable_documentation=off && make -j && make check && make install -j',hosts=nodes,connection_params={'user':'root'})

In [17]:
Remote_install.run().ok

True

#### Version Simgrid

In [18]:
execo.action.Remote(cmd='touch smpi_version.txt && smpicc --version > smpi_version.txt'
                    , hosts=nodes, connection_params={'user': 'root'
                    }).run()
version = ''
for i in range(0, nodecount):
    execo.action.Get(hosts=nodes[i], remote_files=['~/smpi_version.txt'
                     ], local_location='./smpicc_version-'
                     + get_host_shortname(nodes[i]) + '.txt',
                     connection_params={'user': 'root'}).run()
    version = version + execo.Process(cmd='head -n 1 ./smpicc_version-'
            + get_host_shortname(nodes[i]) + '.txt'
            ).run().stdout.replace('\n', '  ')
    execo.Process(cmd='rm ./smpicc_version-'
                  + get_host_shortname(nodes[i]) + '.txt').run()
version


'SimGrid version 3.18  SimGrid version 3.18  SimGrid version 3.18  SimGrid version 3.18  '

In [19]:
path_base = "~/Stage-POLARIS/SimGrid/examples/SMPI/"+experiment_name
path_src = path_base + '/src/'
path_rslt = path_base + '/results/'

#### Récupération fichiers sources

In [ ]:
Src_get = execo.action.Remote(cmd='mkdir -p experiments_src',hosts=nodes,connection_params={'user':'root'})
Src_get.run().ok

In [ ]:
Remote_smpi_env = execo.action.Put(hosts=nodes,local_files=[path_src+"griffon.xml",path_src+"griffon_hostfile.txt",path_src+"broadcast.c",path_src+"Makefile"],remote_location='~/experiments_src',connection_params={'user':'root'})
Remote_smpi_env.run().ok

#### Execution de l'expérience

In [ ]:
Remote_smpi = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS=2 MIN=0 MAX=10 N=2 DELAY=\"0 0\"',hosts=nodes,connection_params={'user':'root'})
Remote_smpi.run().ok

#### Récupération des résultats

In [ ]:
Result_get = \
    execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))
Result_get.run().ok

In [ ]:
for i in range(0, nodecount):
    execo.action.Get(hosts = nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location = path_rslt
                     + datetime.datetime.now().strftime('%Y-%m-%d')
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M'
                     ) + '.txt', connection_params={'user': 'root'
                     }).run().ok

#### Delayer

In [20]:
# Parsing the results
def parser(num_node,date_precise):
    with open('/home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/Time_Bcast/results/'
               + date_precise.split('_')[0] + '/'
              + get_host_shortname(nodes[num_node]) + '_'
              + date_precise
              + '.txt') as f:
        return f.readlines()

In [21]:
# Converting and organizing data
def tab_conv(l,NPROCS) :
    S = init_tab(NPROCS, None)
        
    # Taking useful data    
    for i in range(0,NPROCS) :
        S[int(l[i].split()[1])] = int(float(l[i].split()[5]))
    return S

In [22]:
def delayer(num_node,NPROCS,date):
    D = init_tab(NPROCS,0)
    
    raw_data = parser(num_node,date)
    
    S = tab_conv(raw_data,NPROCS)
    
    for p in range(0, NPROCS):
        D[p] = S[p] - min(S)
    return D

In [ ]:
delayer(1,2,'2018-05-30_12:19')

In [23]:
def init_tab(num, elt) :
    D = [elt] * num
    return D

##### Shortcut

In [23]:
Src_get = execo.action.Remote(cmd='mkdir -p experiments_src',
                              hosts=nodes,
                              connection_params={'user': 'root'})
Remote_smpi_env = execo.action.Put(hosts=nodes, local_files=[path_src
                                   + 'griffon.xml', path_src
                                   + 'griffon_hostfile.txt', path_src
                                   + 'broadcast.c', path_src + 'Makefile'],
                                   remote_location='~/experiments_src',
                                   connection_params={'user': 'root'})
Result_get = \
    execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))
def parl_exec () :
    Result_get.run().ok
    # Nbr pcs
    NPROCS = 10
    
    # Nbr itérations
    N = 100
    
    D = init_tab(NPROCS,0)
    delay = ' '.join(str(e) for e in D)
    date = datetime.datetime.now().strftime('%Y-%m-%d');
    date_precise = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
    print(date_precise)
    
    # TODO //
    for i in range (0,nodecount) :
        MIN = (int)((i*N)/len(nodes))
        MAX = (int)(((i+1)*N)/len(nodes))
        print(get_host_shortname(nodes[i])+" => "+str(i)+" NPROCS = "+str(NPROCS)
             +" MIN = "+str(MIN)
             +" MAX = "+str(MAX)
             +" N = "+str(N)
             +" DELAY = '"+ delay +"'"
            )
        bool = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS='
                                  +str(NPROCS)+' MIN='+str(MIN) 
                                  +' MAX='+str(MAX)+' N='+str(N)
                                  +" DELAY='"+delay
                                  +"'",hosts=nodes[i],connection_params={'user':'root'}).run().ok
        print(bool)
        execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location= path_rslt
                     + date
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + date_precise + '.txt', connection_params={'user': 'root'
                     }).run().ok
        print(delayer(i,NPROCS,date_precise))
    return;

# Remote_smpi = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS=2 MIN=0 MAX=10',hosts=nodes,connection_params={'user':'root'})



Src_get.run().ok
Remote_smpi_env.run().ok

# Remote_smpi.run().ok

parl_exec()

for i in range(0, nodecount):
    execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location=path_rslt
                     + datetime.datetime.now().strftime('%Y-%m-%d')
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M'
                     ) + '.txt', connection_params={'user': 'root'
                     }).run().ok

2018-05-31_11:01
ecotype-44 => 0 NPROCS = 10 MIN = 0 MAX = 25 N = 100 DELAY = '0 0 0 0 0 0 0 0 0 0'
True
[6, 6, 9, 3, 0, 11, 7, 2, 7, 10]
ecotype-47 => 1 NPROCS = 10 MIN = 25 MAX = 50 N = 100 DELAY = '0 0 0 0 0 0 0 0 0 0'
True
[8, 0, 6, 6, 6, 6, 6, 8, 9, 5]
ecotype-7 => 2 NPROCS = 10 MIN = 50 MAX = 75 N = 100 DELAY = '0 0 0 0 0 0 0 0 0 0'
True
[6, 0, 6, 6, 6, 8, 9, 7, 8, 4]
ecotype-9 => 3 NPROCS = 10 MIN = 75 MAX = 100 N = 100 DELAY = '0 0 0 0 0 0 0 0 0 0'
True
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


#### //

In [26]:
import random
import sys
from threading import Thread
import time


Src_get = execo.action.Remote(cmd='mkdir -p experiments_src',
                              hosts=nodes,
                              connection_params={'user': 'root'})
Remote_smpi_env = execo.action.Put(hosts=nodes, local_files=[path_src
                                   + 'griffon.xml', path_src
                                   + 'griffon_hostfile.txt', path_src
                                   + 'broadcast.c', path_src + 'Makefile'],
                                   remote_location='~/experiments_src',
                                   connection_params={'user': 'root'})
Result_get = \
    execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))



Src_get.run().ok
Remote_smpi_env.run().ok


machines = init_tab(len(nodes),None)

class ParSimer(Thread):

    """Thread chargé simplement d'afficher une lettre dans la console."""

    def __init__(self, MIN,MAX,N,DELAY):
        Thread.__init__(self)
        self.MIN = MIN
        self.MAX = MAX
        self.N = N
        self.DELAY = DELAY
        
    def run(self):
        """Code à exécuter pendant l'exécution du thread."""
        print(get_host_shortname(nodes[i])+" => "+str(i)+" NPROCS = "+str(NPROCS)
             +" MIN = "+str(self.MIN)
             +" MAX = "+str(self.MAX)
             +" N = "+str(self.N)
             +" DELAY = '"+ self.DELAY +"'"
            )
        bool = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS='
                                  +str(NPROCS)+' MIN='+str(self.MIN) 
                                  +' MAX='+str(self.MAX)+' N='+str(self.N)
                                  +" DELAY='"+self.DELAY
                                  +"'",hosts=nodes[i],connection_params={'user':'root'}).run().ok
        print(bool)
       
                      
            
# Création des threads
Result_get.run().ok
# Nbr pcs
NPROCS = 10
    
# Nbr itérations
N = 100
    
D = init_tab(NPROCS,0)
delay = ' '.join(str(e) for e in D)
date = datetime.datetime.now().strftime('%Y-%m-%d');
date_precise = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')



for i in range (0,nodecount) :
    MIN = (int)((i*N)/len(nodes))
    MAX = (int)(((i+1)*N)/len(nodes)) 
    machines[i] = ParSimer(MIN,MAX,N,delay)

# Lancement des threads
for i in range (0,nodecount) :
    machines[i].start()

for i in range (0,nodecount) :
    machines[i].join()
    execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location= path_rslt
                     + date
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + date_precise + '.txt', connection_params={'user': 'root'
                     }).run().ok     
    print(delayer(i,NPROCS,date_precise))

parasilo-26 => 0 NPROCS = 10 MIN = 0 MAX = 25 N = 100 DELAY = '0 0 0 0 0 0 0 0 0 0'
parasilo-27 => 1 NPROCS = 10 MIN = 25 MAX = 50 N = 100 DELAY = '0 0 0 0 0 0 0 0 0 0'
parasilo-28 => 2 NPROCS = 10 MIN = 50 MAX = 75 N = 100 DELAY = '0 0 0 0 0 0 0 0 0 0'
parasilo-9 => 3 NPROCS = 10 MIN = 75 MAX = 100 N = 100 DELAY = '0 0 0 0 0 0 0 0 0 0'
True
True
True
True
[2, 1, 0, 1, 10, 5, 5, 3, 3, 5]
[1, 7, 0, 1, 7, 2, 3, 3, 4, 7]
[4, 1, 1, 5, 2, 1, 0, 2, 4, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


#### Publication des résultats

In [ ]:
%%bash -s $experiment_name $site
git add .
git commit -m 'Automatic results update : '"$1"'---'"$2"' Delayer'

#### Fin d'expérience : suppression du job

In [10]:
oardel(jobid)

TypeError: 'int' object is not iterable